In [2]:
#!/usr/bin/env python3
"""
==========================================================================
 GRAMMATICAL EVOLUTION: INITIALIZATION METHODS COMPARISON
 GRAPE-faithful core · Sequential execution · 4 benchmarks

 GE core (Grammar, mapper, Individual, operators) faithfully reproduces
 the BDS Group's GRAPE codebase: https://github.com/bdsul/grape
 (de Lima et al., 2022, Signals 3(3), 642-663)

 Requirements:
   pip install numpy scipy matplotlib scikit-learn

 Usage:
   python ge_experiment_grape.py

 References:
   [1] Ryan & Azad (2003) Sensible Initialisation in GE, GECCO
   [2] Nicolau (2017) Understanding GE: Initialisation, GPEM 18(4)
   [3] de Lima et al. (2022) GRAPE, Signals 3(3), 642-663
   [4] Murphy et al. (2024) Structured GE Initialisation, GPEM 25(2)
==========================================================================
"""

import numpy as np
import random
import re
import math
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.stats import mannwhitneyu
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import warnings
import time
import json
from datetime import datetime

warnings.filterwarnings('ignore')

SEED = 42

print("=" * 70)
print(" GRAMMATICAL EVOLUTION: INITIALIZATION METHODS COMPARISON")
print(" (GRAPE-faithful core · Sequential · 4 benchmarks)")
print("=" * 70)
print(f"Experiment date : {datetime.now().strftime('%Y-%m-%d %H:%M')}")


# ==========================================================================
#  1. PROTECTED FUNCTIONS — exact copies from GRAPE functions.py
# ==========================================================================

def pdiv(a, b):
    """Protected division (GRAPE functions.py)."""
    try:
        with np.errstate(divide='ignore', invalid='ignore'):
            return np.where(b == 0, np.ones_like(a), a / b)
    except ZeroDivisionError:
        return 1.0


def plog(a):
    """Protected log: log(1+|a|) (GRAPE functions.py)."""
    return np.log(1.0 + np.abs(a))


def psqrt(a):
    """Protected sqrt: sqrt(|a|) (GRAPE functions.py)."""
    return np.sqrt(abs(a))


def exp(a):
    """Exponential with overflow protection."""
    return np.exp(np.clip(a, -500, 500))


# ==========================================================================
#  2. GRAMMAR — reproducing GRAPE's Grammar class (grape.py)
#
#  Each production rule stored as:
#    [0] string, [1] 'terminal'/'non-terminal', [2] arity,
#    [3] production choice label, [4] recursive (bool),
#    [5] min depth to terminate
# ==========================================================================

class Grammar:
    """Faithful reproduction of GRAPE's Grammar class."""

    def __init__(self, bnf_text):
        bnf_grammar = re.sub(r"\s+", " ", bnf_text)

        self.non_terminals = [
            '<' + t + '>'
            for t in re.findall(r"\<([\(\)\w,\-.]+)\>\s*::=", bnf_grammar)
        ]
        self.start_rule = self.non_terminals[0]

        for nt in self.non_terminals:
            bnf_grammar = bnf_grammar.replace(nt + " ::=", "  ::=")
        rules = bnf_grammar.split("::=")[1:]
        rules = [r.replace('\n', '').replace('\t', '') for r in rules]

        self.production_rules = [r.split('|') for r in rules]
        for i in range(len(self.production_rules)):
            self.production_rules[i] = [
                p.strip() for p in self.production_rules[i]
            ]
            for j in range(len(self.production_rules[i])):
                pr_str = self.production_rules[i][j]
                nts_in = re.findall(r"\<([\(\)\w,\-.]+)\>", pr_str)
                if nts_in:
                    self.production_rules[i][j] = [
                        pr_str, "non-terminal", len(nts_in), j
                    ]
                else:
                    self.production_rules[i][j] = [
                        pr_str, "terminal", 0, j
                    ]

        self.n_rules = [len(lst) for lst in self.production_rules]

        # --- Recursiveness check (exact GRAPE logic) ---
        for i in range(len(self.production_rules)):
            for j in range(len(self.production_rules[i])):
                nts = [
                    '<' + t + '>'
                    for t in re.findall(
                        r"\<([\(\)\w,\-.]+)\>",
                        self.production_rules[i][j][0]
                    )
                ]
                recursive = False
                for nt_c in list(dict.fromkeys(nts)):
                    stack = [self.non_terminals[i]]
                    if nt_c in stack:
                        recursive = True
                        break
                    stack.append(nt_c)
                    recursive = self._check_rec(nt_c, stack)
                    if recursive:
                        break
                    stack.pop()
                self.production_rules[i][j].append(recursive)  # index [4]

        # --- Minimum depth to terminate (exact GRAPE logic) ---
        nt_depth = [None] * len(self.non_terminals)
        part_depth = []
        isolated_nt = []
        for i in range(len(self.production_rules)):
            part_depth.append([])
            isolated_nt.append([])
            for j in range(len(self.production_rules[i])):
                part_depth[i].append([])
                isolated_nt[i].append([])
                if self.production_rules[i][j][1] == 'terminal':
                    isolated_nt[i][j].append(None)
                    part_depth[i][j] = 1
                    if not nt_depth[i]:
                        nt_depth[i] = 1
                else:
                    for k in range(self.production_rules[i][j][2]):
                        part_depth[i][j].append([])
                        t = re.findall(
                            r"\<([\(\)\w,\-.]+)\>",
                            self.production_rules[i][j][0]
                        )[k]
                        isolated_nt[i][j].append('<' + t + '>')

        cont = True
        while cont:
            if None not in nt_depth:
                cont = False
            for i in range(len(self.non_terminals)):
                for j in range(len(self.production_rules)):
                    for k in range(len(self.production_rules[j])):
                        for l in range(len(isolated_nt[j][k])):
                            if self.non_terminals[i] == isolated_nt[j][k][l]:
                                if nt_depth[i] and not part_depth[j][k][l]:
                                    part_depth[j][k][l] = nt_depth[i] + 1
                                    if ([] not in part_depth[j][k]
                                            and not nt_depth[j]):
                                        nt_depth[j] = part_depth[j][k][l]

        for i in range(len(part_depth)):
            for j in range(len(part_depth[i])):
                d = (part_depth[i][j]
                     if isinstance(part_depth[i][j], int)
                     else max(part_depth[i][j]))
                self.production_rules[i][j].append(d)  # index [5]

    def _check_rec(self, nt, stack):
        """Exact reproduction of GRAPE's check_recursiveness()."""
        idx = self.non_terminals.index(nt)
        for j in range(len(self.production_rules[idx])):
            nts = [
                '<' + t + '>'
                for t in re.findall(
                    r"\<([\(\)\w,\-.]+)\>",
                    self.production_rules[idx][j][0]
                )
            ]
            for nc in list(dict.fromkeys(nts)):
                if nc in stack:
                    return True
                stack.append(nc)
                if self._check_rec(nc, stack):
                    return True
                stack.pop()
        return False


# ==========================================================================
#  3. MAPPER — lazy codon consumption (GRAPE default)
# ==========================================================================

def mapper_lazy(genome, grammar, max_depth):
    """
    Lazy mapper: only consumes a codon when there are >1 production choices
    for the current non-terminal. This is the GRAPE default.
    """
    idx_genome = 0
    phenotype = grammar.start_rule
    nxt = re.search(r"\<([\(\)\w,\-.]+)\>", phenotype)
    if not nxt:
        return phenotype, 0, 1, 0, True, 0, []

    next_NT = nxt.group()
    n_start = len(re.findall(r"\<([\(\)\w,\-.]+)\>", phenotype))
    list_depth = [1] * n_start
    idx_depth = 0
    nodes = 0
    structure = []

    while next_NT and idx_genome < len(genome):
        NT_i = grammar.non_terminals.index(next_NT)
        n_opts = grammar.n_rules[NT_i]

        if n_opts > 1:
            idx_prod = genome[idx_genome] % n_opts
            idx_genome += 1
        else:
            idx_prod = 0

        pr = grammar.production_rules[NT_i][idx_prod]
        structure.append(pr[3])
        phenotype = phenotype.replace(next_NT, pr[0], 1)
        list_depth[idx_depth] += 1

        if list_depth[idx_depth] > max_depth:
            break

        if pr[2] == 0:  # terminal
            idx_depth += 1
            nodes += 1
        elif pr[2] > 1:  # arity > 1
            ar = pr[2]
            if idx_depth == 0:
                list_depth = [list_depth[0]] * ar + list_depth[1:]
            else:
                list_depth = (list_depth[:idx_depth]
                              + [list_depth[idx_depth]] * ar
                              + list_depth[idx_depth + 1:])

        nxt = re.search(r"\<([\(\)\w,\-.]+)\>", phenotype)
        next_NT = nxt.group() if nxt else None

    if next_NT:
        return phenotype, nodes, max(list_depth), 0, True, 0, structure
    return phenotype, nodes, max(list_depth), idx_genome, False, 0, structure


# ==========================================================================
#  4. INDIVIDUAL
# ==========================================================================

class Individual:
    """GE individual with genome-to-phenotype mapping."""
    __slots__ = [
        'genome', 'phenotype', 'nodes', 'depth',
        'used_codons', 'invalid', 'n_wraps', 'structure', 'fitness'
    ]

    def __init__(self, genome, grammar, max_depth):
        self.genome = genome
        (self.phenotype, self.nodes, self.depth,
         self.used_codons, self.invalid, self.n_wraps,
         self.structure) = mapper_lazy(genome, grammar, max_depth)
        self.fitness = None


# ==========================================================================
#  5. INITIALISATION — exact GRAPE logic (grape.py)
# ==========================================================================

def random_initialisation(pop_size, grammar, min_gl, max_gl,
                          max_depth, codon_size):
    """
    Random initialisation: genome of random length [min_gl, max_gl]
    with random integer codons in [0, codon_size].
    """
    pop = []
    for _ in range(pop_size):
        gl = random.randint(min_gl, max_gl)
        genome = [random.randint(0, codon_size) for _ in range(gl)]
        pop.append(Individual(genome, grammar, max_depth))
    return pop


def sensible_initialisation(pop_size, grammar, min_d, max_d, codon_size):
    """
    Sensible initialisation with Ramped Half-and-Half (RHH).
    Half Grow (ramped across depths), half Full.
    """
    is_odd = pop_size % 2
    n_grow = pop_size // 2
    n_sets = max_d - min_d + 1
    set_sz = n_grow // n_sets
    remaining = n_grow % n_sets
    n_full = n_grow + is_odd + remaining

    pop = []

    # Grow (ramped)
    for i in range(n_sets):
        md = min_d + i
        for _ in range(set_sz):
            pop.append(_build_tree(grammar, md, codon_size, 'grow'))

    # Full
    for _ in range(n_full):
        pop.append(_build_tree(grammar, max_d, codon_size, 'full'))

    return pop


def _build_tree(grammar, max_depth, codon_size, method):
    """Build one individual using grow or full method (GRAPE style)."""
    remainders = []
    possible_choices = []

    phenotype = grammar.start_rule
    rem_NTs = [
        '<' + t + '>'
        for t in re.findall(r"\<([\(\)\w,\-.]+)\>", phenotype)
    ]
    depths = [1] * len(rem_NTs)
    idx_b = 0

    while rem_NTs:
        idx_NT = grammar.non_terminals.index(rem_NTs[0])
        total = grammar.production_rules[idx_NT]
        actual = [pr for pr in total
                  if pr[5] + depths[idx_b] <= max_depth]

        if not actual:
            actual = [pr for pr in total if pr[1] == 'terminal']
            if not actual:
                actual = total

        if method == 'full':
            rec = [pr for pr in actual if pr[4]]
            ch = random.choice(rec) if rec else random.choice(actual)
        else:
            ch = random.choice(actual)

        phenotype = phenotype.replace(rem_NTs[0], ch[0], 1)
        depths[idx_b] += 1

        if len(total) > 1:
            remainders.append(ch[3])
            possible_choices.append(len(total))

        if ch[2] > 1:
            ar = ch[2]
            if idx_b == 0:
                depths = [depths[0]] * ar + depths[1:]
            else:
                depths = (depths[:idx_b]
                          + [depths[idx_b]] * ar
                          + depths[idx_b + 1:])

        if ch[1] == 'terminal':
            idx_b += 1

        rem_NTs = [
            '<' + t + '>'
            for t in re.findall(r"\<([\(\)\w,\-.]+)\>", phenotype)
        ]

    genome = []
    for k in range(len(remainders)):
        codon = (
            random.randint(0, int(1e10))
            % math.floor((codon_size + 1) / possible_choices[k])
            * possible_choices[k]
        ) + remainders[k]
        genome.append(codon)

    tail = max(int(0.5 * len(genome)), 1)
    for _ in range(tail):
        genome.append(random.randint(0, codon_size))

    return Individual(genome, grammar, max_depth)


# ==========================================================================
#  6. GENETIC OPERATORS
# ==========================================================================

def crossover_onepoint(p0, p1, grammar, max_depth):
    """One-point crossover within the effective genome (GRAPE style)."""
    cx0 = max(1, min(len(p0.genome), p0.used_codons)
              if not p0.invalid else len(p0.genome))
    cx1 = max(1, min(len(p1.genome), p1.used_codons)
              if not p1.invalid else len(p1.genome))

    for _ in range(20):
        pt0 = random.randint(1, cx0)
        pt1 = random.randint(1, cx1)
        g0 = p0.genome[:pt0] + p1.genome[pt1:]
        g1 = p1.genome[:pt1] + p0.genome[pt0:]
        c0 = Individual(g0, grammar, max_depth)
        c1 = Individual(g1, grammar, max_depth)
        if c0.depth <= max_depth and c1.depth <= max_depth:
            return c0, c1

    return (Individual(p0.genome[:], grammar, max_depth),
            Individual(p1.genome[:], grammar, max_depth))


def mutation_int_flip(ind, mut_pb, codon_size, grammar, max_depth):
    """Per-codon integer flip mutation (GRAPE style)."""
    g = ind.genome[:]
    for i in range(len(g)):
        if random.random() < mut_pb:
            g[i] = random.randint(0, codon_size)
    return Individual(g, grammar, max_depth)


def tournament_sel(pop, k, ts):
    """Standard tournament selection."""
    sel = []
    for _ in range(k):
        asp = random.sample(pop, min(ts, len(pop)))
        sel.append(min(
            asp,
            key=lambda x: (x.fitness
                           if x.fitness is not None and not np.isnan(x.fitness)
                           else float('inf'))
        ))
    return sel


print("GE library loaded")


# ==========================================================================
#  7. EXPERIMENTAL PARAMETERS (GRAPE defaults)
# ==========================================================================

POP_SIZE       = 200
MAX_GENS       = 200
P_CX           = 0.8
P_MUT          = 0.01
ELITE          = 0
TOURN          = 7

MIN_INIT_GL    = 30    # Random init genome length
MAX_INIT_GL    = 50    # Random init genome length
MAX_INIT_DEPTH = 13    # Sensible init max depth
MIN_INIT_DEPTH = 3     # Sensible init min depth
MAX_TREE_DEPTH = 35    # Runtime depth limit
CODON_SIZE     = 255

N_RUNS         = 30

print("=" * 70)
print(" EXPERIMENTAL PARAMETERS (GRAPE)")
print("=" * 70)
print(f"  Runs           : {N_RUNS}")
print(f"  Generations    : {MAX_GENS}")
print(f"  Population     : {POP_SIZE}")
print(f"  Elite Size     : {ELITE}")
print(f"  Tournament     : {TOURN}")
print(f"  Crossover      : {P_CX}")
print(f"  Mutation Rate  : {P_MUT}")
print(f"  Max Tree Depth : {MAX_TREE_DEPTH}")
print(f"  Codon Size     : {CODON_SIZE}")
print(f"  Random genome  : [{MIN_INIT_GL}, {MAX_INIT_GL}]")
print(f"  Sensible depth : [{MIN_INIT_DEPTH}, {MAX_INIT_DEPTH}]")
print(f"  Mapper         : Lazy codon consumption")
print(f"  Fitness        : MSE")
print("=" * 70)

COLORS = {'Random': '#E74C3C', 'Sensible': '#3498DB'}


# ==========================================================================
#  8. GRAMMARS — GRAPE-style BNF (x[i] indexing)
# ==========================================================================

def make_grammar_bnf(n_features):
    """
    Build a GRAPE-style BNF grammar string for the given dimensionality.
    Uses x[0], x[1], ... notation and GRAPE protected operators.
    """
    vars_list = ' | '.join([f'x[{i}]' for i in range(n_features)])

    bnf = (
        "<e> ::= <e>+<e> | <e>-<e> | <e>*<e> | pdiv(<e>,<e>) | "
        "psqrt(<e>) | np.sin(<e>) | np.tanh(<e>) | plog(<e>) | "
        f"{vars_list} | <c><c>.<c><c>\n"
        "<c> ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9"
    )
    return bnf


# ==========================================================================
#  9. BENCHMARK PROBLEMS
#     Data stored in GRAPE format: X shape = (n_features, n_samples)
# ==========================================================================

def load_keijzer6():
    """Keijzer-6: sum(1/i, i=1..x), univariate."""
    def keijzer_6(x):
        return np.array([sum(1.0 / i for i in range(1, int(xi) + 1))
                         for xi in x])

    x_tr = np.linspace(1, 50, 50)
    y_tr = keijzer_6(x_tr)
    x_te = np.linspace(1, 120, 120)
    y_te = keijzer_6(x_te)
    # GRAPE format: (n_features, n_samples)
    return np.array([x_tr]), y_tr, np.array([x_te]), y_te


def load_nguyen7():
    """Nguyen-7: ln(x+1) + ln(x²+1), univariate."""
    x_tr = np.linspace(0, 2, 20)
    y_tr = np.log(x_tr + 1) + np.log(x_tr ** 2 + 1)
    x_te = np.linspace(0, 2, 100)
    y_te = np.log(x_te + 1) + np.log(x_te ** 2 + 1)
    return np.array([x_tr]), y_tr, np.array([x_te]), y_te


def load_pagie1():
    """Pagie-1: 1/(1+x^-4) + 1/(1+y^-4), 2D grid."""
    vals = np.linspace(-5, 5, 26)  # 676 training points
    X0, X1 = np.meshgrid(vals, vals)
    x0, x1 = X0.ravel(), X1.ravel()
    with np.errstate(divide='ignore', invalid='ignore'):
        y = (1.0 / (1.0 + np.power(np.abs(x0) + 1e-10, -4))
             + 1.0 / (1.0 + np.power(np.abs(x1) + 1e-10, -4)))
    y = np.nan_to_num(y, nan=0.0, posinf=2.0, neginf=0.0)

    rng = np.random.RandomState(SEED)
    x0t = rng.uniform(-5, 5, 10000)
    x1t = rng.uniform(-5, 5, 10000)
    with np.errstate(divide='ignore', invalid='ignore'):
        yt = (1.0 / (1.0 + np.power(np.abs(x0t) + 1e-10, -4))
              + 1.0 / (1.0 + np.power(np.abs(x1t) + 1e-10, -4)))
    yt = np.nan_to_num(yt, nan=0.0, posinf=2.0, neginf=0.0)
    return np.array([x0, x1]), y, np.array([x0t, x1t]), yt


def load_diabetes():
    """Sklearn Diabetes dataset (10 features)."""
    from sklearn.datasets import load_diabetes as _load
    data = _load()
    X, y = data.data, data.target
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.3, random_state=SEED)
    # Transpose to GRAPE format: (n_features, n_samples)
    return X_tr.T, y_tr, X_te.T, y_te


PROBLEMS = OrderedDict([
    ('Keijzer-6', {
        'loader': load_keijzer6,
        'n_features': 1,
        'desc': 'Harmonic series (1D)',
    }),
    ('Nguyen-7', {
        'loader': load_nguyen7,
        'n_features': 1,
        'desc': 'ln(x+1)+ln(x²+1) (1D)',
    }),
    ('Pagie-1', {
        'loader': load_pagie1,
        'n_features': 2,
        'desc': '1/(1+x^-4)+1/(1+y^-4) (2D)',
    }),
    ('Diabetes', {
        'loader': load_diabetes,
        'n_features': 10,
        'desc': 'Sklearn benchmark (10D)',
    }),
])

print("\nBenchmark Problems:")
for name, info in PROBLEMS.items():
    print(f"  {name}: {info['desc']}")


# ==========================================================================
# 10. FITNESS EVALUATION — MSE (GRAPE style)
# ==========================================================================

_EVAL_GLOBALS = {
    'np': np, 'pdiv': pdiv, 'plog': plog, 'psqrt': psqrt, 'exp': exp,
    '__builtins__': {},
}


def fitness_eval(ind, x, y):
    """Evaluate fitness as MSE. Returns np.nan if invalid or error."""
    if ind.invalid:
        return np.nan

    try:
        pred = eval(ind.phenotype, _EVAL_GLOBALS, {'x': x})
    except Exception:
        return np.nan

    if not np.isrealobj(pred):
        return np.nan

    try:
        f = float(np.mean(np.square(y - pred)))
    except Exception:
        return np.nan

    if f == float('inf') or np.isnan(f):
        return np.nan
    return f


# ==========================================================================
# 11. SINGLE EVOLUTIONARY RUN
# ==========================================================================

def run_ge(init_method, grammar, X_tr, Y_tr, X_te, Y_te, seed):
    """Run one full GE evolution and return metrics."""
    random.seed(seed)

    # --- Initialisation ---
    if init_method == 'Random':
        pop = random_initialisation(
            POP_SIZE, grammar, MIN_INIT_GL, MAX_INIT_GL,
            MAX_TREE_DEPTH, CODON_SIZE
        )
    else:
        pop = sensible_initialisation(
            POP_SIZE, grammar, MIN_INIT_DEPTH, MAX_INIT_DEPTH, CODON_SIZE
        )

    # Evaluate initial population
    for ind in pop:
        ind.fitness = fitness_eval(ind, X_tr, Y_tr)

    # --- Initial stats ---
    valid = [i for i in pop
             if not i.invalid
             and i.fitness is not None
             and not np.isnan(i.fitness)]

    init_stats = {
        'validity': len(valid) / POP_SIZE,
        'best_mse': float(min(i.fitness for i in valid)) if valid else np.nan,
        'mean_mse': float(np.mean([i.fitness for i in valid])) if valid else np.nan,
        'mean_depth': float(np.mean([i.depth for i in valid])) if valid else 0,
        'mean_nodes': float(np.mean([i.nodes for i in valid])) if valid else 0,
        'mean_gl': float(np.mean([len(i.genome) for i in valid])) if valid else 0,
        'unique_pheno': len(set(i.phenotype for i in valid if i.phenotype)),
        'struct_div': (len(set(tuple(i.structure) for i in valid))
                       / max(len(valid), 1)) if valid else 0,
    }

    # --- History ---
    hist = {
        'min': [], 'avg': [], 'invalid': [], 'fitness_test': [],
        'avg_depth': [], 'avg_nodes': [], 'struct_div': [],
    }

    hof = None  # Hall of fame (best ever)

    # --- Evolution loop ---
    for gen in range(MAX_GENS + 1):
        vp = [i for i in pop
              if not i.invalid
              and i.fitness is not None
              and not np.isnan(i.fitness)]
        fits = [i.fitness for i in vp]

        # Update hall of fame
        best = min(vp, key=lambda x: x.fitness) if vp else None
        if best and (hof is None
                     or best.fitness is not None
                     and not np.isnan(best.fitness)
                     and (hof.fitness is None
                          or np.isnan(hof.fitness)
                          or best.fitness < hof.fitness)):
            hof = Individual(best.genome[:], grammar, MAX_TREE_DEPTH)
            hof.fitness = best.fitness

        # Test fitness of best
        test_f = (fitness_eval(hof, X_te, Y_te)
                  if hof and not hof.invalid else np.nan)

        # Log generation stats
        hist['min'].append(float(min(fits)) if fits else np.nan)
        hist['avg'].append(float(np.mean(fits)) if fits else np.nan)
        hist['invalid'].append(POP_SIZE - len(vp))
        hist['fitness_test'].append(
            float(test_f) if not np.isnan(test_f) else np.nan
        )
        hist['avg_depth'].append(
            float(np.mean([i.depth for i in vp])) if vp else 0
        )
        hist['avg_nodes'].append(
            float(np.mean([i.nodes for i in vp])) if vp else 0
        )
        hist['struct_div'].append(
            len(set(tuple(i.structure) for i in vp))
            / max(len(vp), 1) if vp else 0
        )

        if gen == MAX_GENS:
            break

        # --- Selection ---
        sel = tournament_sel(pop, POP_SIZE, TOURN)

        # --- Crossover ---
        off = []
        for i in range(0, len(sel) - 1, 2):
            if random.random() < P_CX:
                c0, c1 = crossover_onepoint(
                    sel[i], sel[i + 1], grammar, MAX_TREE_DEPTH
                )
            else:
                c0 = Individual(sel[i].genome[:], grammar, MAX_TREE_DEPTH)
                c1 = Individual(sel[i + 1].genome[:], grammar, MAX_TREE_DEPTH)
            off.extend([c0, c1])
        if len(sel) % 2 == 1:
            off.append(
                Individual(sel[-1].genome[:], grammar, MAX_TREE_DEPTH)
            )

        # --- Mutation ---
        mut = [
            mutation_int_flip(i, P_MUT, CODON_SIZE, grammar, MAX_TREE_DEPTH)
            for i in off
        ]

        # --- Evaluate ---
        for i in mut:
            i.fitness = fitness_eval(i, X_tr, Y_tr)

        pop = mut

    # Final test evaluation
    ft = (fitness_eval(hof, X_te, Y_te)
          if hof and not hof.invalid else np.nan)

    return {
        'init': init_stats,
        'hist': hist,
        'train_mse': hof.fitness if hof else np.nan,
        'test_mse': float(ft),
        'pheno': hof.phenotype if hof else None,
        'depth': hof.depth if hof else 0,
        'nodes': hof.nodes if hof else 0,
        'gl': len(hof.genome) if hof else 0,
    }


# ==========================================================================
# 12. RUN EXPERIMENT — SEQUENTIAL
# ==========================================================================

def main():
    print("\n" + "=" * 70)
    print(f" RUNNING EXPERIMENT (Sequential)")
    print(f" {len(PROBLEMS)} Problems × 2 Methods × {N_RUNS} Runs"
          f" = {len(PROBLEMS)*2*N_RUNS} jobs")
    print("=" * 70)

    all_results = {}
    total_start = time.time()

    for prob_name, prob_info in PROBLEMS.items():
        print(f"\n{'#' * 60}")
        print(f"# {prob_name}: {prob_info['desc']}")
        print(f"{'#' * 60}")

        bnf = make_grammar_bnf(prob_info['n_features'])
        grammar = Grammar(bnf)

        print(f"  Grammar: {len(grammar.non_terminals)} NTs, "
              f"{sum(grammar.n_rules)} total productions")
        for i, nt in enumerate(grammar.non_terminals):
            n = grammar.n_rules[i]
            rec = sum(1 for p in grammar.production_rules[i] if p[4])
            print(f"    {nt}: {n} prods "
                  f"(recursive={rec}, non-recursive={n - rec})")

        problem_results = {}

        for method_name in ['Random', 'Sensible']:
            t0 = time.time()
            runs = []
            for r in range(N_RUNS):
                if (r + 1) % 5 == 0 or r == 0:
                    print(f"  {method_name} run {r + 1}/{N_RUNS}...",
                          flush=True)

                np.random.seed(SEED + r)
                X_tr, Y_tr, X_te, Y_te = prob_info['loader']()
                result = run_ge(
                    method_name, grammar, X_tr, Y_tr, X_te, Y_te,
                    SEED + r
                )
                runs.append(result)

            elapsed = time.time() - t0
            print(f"  {method_name}: {elapsed:.0f}s "
                  f"({elapsed / N_RUNS:.1f}s/run)")
            problem_results[method_name] = runs

        all_results[prob_name] = problem_results

    total_t = time.time() - total_start
    print(f"\n{'=' * 70}")
    print(f" ALL RUNS COMPLETED in {total_t / 60:.1f} minutes")
    print(f"{'=' * 70}")

    # ==================================================================
    # 13. STATISTICAL ANALYSIS
    # ==================================================================

    def sig_stars(p):
        if p < 0.001:
            return "***"
        elif p < 0.01:
            return "**"
        elif p < 0.05:
            return "*"
        return "ns"

    def vda(a, b):
        """Vargha-Delaney A effect size."""
        m, n = len(a), len(b)
        if m == 0 or n == 0:
            return 0.5
        return sum(
            1 if ai < bi else 0.5 if ai == bi else 0
            for ai in a for bi in b
        ) / (m * n)

    def effect_label(a):
        d = abs(a - 0.5)
        if d < 0.06:
            return "negl"
        elif d < 0.14:
            return "small"
        elif d < 0.21:
            return "medium"
        return "large"

    def safe_vals(lst):
        return [x for x in lst
                if x is not None and not np.isnan(x) and x < 1e10]

    # --- Per-problem detailed analysis ---
    for pn, res in all_results.items():
        print(f"\n{'=' * 70}")
        print(f" {pn}")
        print(f"{'=' * 70}")

        metrics = [
            ("Init Validity %",
             lambda r: r['init']['validity'] * 100),
            ("Init Unique Pheno",
             lambda r: r['init']['unique_pheno']),
            ("Init Mean Depth",
             lambda r: r['init']['mean_depth']),
            ("Init Mean Nodes",
             lambda r: r['init']['mean_nodes']),
            ("Init Mean GenomeLen",
             lambda r: r['init']['mean_gl']),
            ("Init Struct Diversity",
             lambda r: r['init']['struct_div']),
            ("Init Best MSE",
             lambda r: r['init']['best_mse']),
            ("Init Mean MSE",
             lambda r: r['init']['mean_mse']),
            ("Final Train MSE",
             lambda r: r['train_mse']),
            ("Final Test MSE",
             lambda r: r['test_mse']),
        ]

        for label, extract in metrics:
            rv = safe_vals([extract(r) for r in res['Random']])
            svl = safe_vals([extract(r) for r in res['Sensible']])
            if rv and svl:
                _, p = mannwhitneyu(rv, svl, alternative='two-sided')
                a = vda(rv, svl)
                print(
                    f"  {label:<22}: "
                    f"Rand={np.mean(rv):>10.4f}+-{np.std(rv):<8.4f}  "
                    f"Sens={np.mean(svl):>10.4f}+-{np.std(svl):<8.4f}  "
                    f"p={p:.4f}({sig_stars(p)}) A={a:.3f}({effect_label(a)})"
                )
            else:
                print(f"  {label:<22}: insufficient data")

        print(f"\n  Best solutions found:")
        for m in ['Random', 'Sensible']:
            v = [r for r in res[m]
                 if r['train_mse'] is not None
                 and not np.isnan(r['train_mse'])]
            if v:
                b = min(v, key=lambda r: r['train_mse'])
                ph = (b['pheno'] or 'N/A')[:120]
                print(f"    {m}: Train MSE={b['train_mse']:.6f}  "
                      f"Test MSE={b['test_mse']:.6f}  "
                      f"Depth={b['depth']}  Nodes={b['nodes']}")
                print(f"      {ph}")

    # --- Comprehensive summary tables ---
    print("\n" + "=" * 80)
    print(" COMPREHENSIVE RESULTS SUMMARY")
    print("=" * 80)

    prob_names = list(all_results.keys())

    print("\n" + "-" * 80)
    print(" INITIAL POPULATION VALIDITY (%)")
    print("-" * 80)
    print(f"{'Problem':<15} {'Random':>15} {'Sensible':>15} {'p-value':>12} {'Sig':>6}")
    print("-" * 80)
    for p in prob_names:
        rv = safe_vals([r['init']['validity'] * 100
                        for r in all_results[p]['Random']])
        sv = safe_vals([r['init']['validity'] * 100
                        for r in all_results[p]['Sensible']])
        if rv and sv:
            _, pval = mannwhitneyu(rv, sv)
            sig = sig_stars(pval)
            print(f"{p:<15} {np.mean(rv):>6.1f} +- {np.std(rv):>4.1f} "
                  f"{np.mean(sv):>6.1f} +- {np.std(sv):>4.1f} "
                  f"{pval:>12.6f} {sig:>6}")

    print("\n" + "-" * 80)
    print(" FINAL TRAIN MSE")
    print("-" * 80)
    print(f"{'Problem':<15} {'Random':>15} {'Sensible':>15} {'p-value':>12} {'Sig':>6}")
    print("-" * 80)
    for p in prob_names:
        rv = safe_vals([r['train_mse'] for r in all_results[p]['Random']])
        sv = safe_vals([r['train_mse'] for r in all_results[p]['Sensible']])
        if rv and sv:
            _, pval = mannwhitneyu(rv, sv)
            sig = sig_stars(pval)
            print(f"{p:<15} {np.mean(rv):>6.2f} +- {np.std(rv):>4.2f} "
                  f"{np.mean(sv):>6.2f} +- {np.std(sv):>4.2f} "
                  f"{pval:>12.6f} {sig:>6}")

    print("\n" + "-" * 80)
    print(" FINAL TEST MSE")
    print("-" * 80)
    print(f"{'Problem':<15} {'Random':>15} {'Sensible':>15} {'p-value':>12} {'Sig':>6}")
    print("-" * 80)
    for p in prob_names:
        rv = safe_vals([r['test_mse'] for r in all_results[p]['Random']])
        sv = safe_vals([r['test_mse'] for r in all_results[p]['Sensible']])
        if rv and sv:
            _, pval = mannwhitneyu(rv, sv)
            sig = sig_stars(pval)
            print(f"{p:<15} {np.mean(rv):>6.2f} +- {np.std(rv):>4.2f} "
                  f"{np.mean(sv):>6.2f} +- {np.std(sv):>4.2f} "
                  f"{pval:>12.6f} {sig:>6}")

    print("\n" + "=" * 80)
    print(" Significance: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")
    print("=" * 80)

    # ==================================================================
    # 14. PLOTS
    # ==================================================================

    plt.rcParams.update({
        'font.size': 10,
        'axes.titlesize': 11,
        'axes.labelsize': 10,
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'savefig.bbox': 'tight',
        'axes.grid': True,
        'grid.alpha': 0.3,
        'lines.linewidth': 1.5,
    })
    n_probs = len(PROBLEMS)

    # --- Fig 1: Initial Population Quality (2×2 grid) ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(
        'Initial Population Quality: Random vs Sensible (RHH)\n'
        '[GRAPE parameters: Pop=200, Lazy mapper, CodonSize=255]',
        fontsize=13, fontweight='bold'
    )

    for idx, (pn, res) in enumerate(all_results.items()):
        ax = axes[idx // 2, idx % 2]
        d = {m: safe_vals([r['init']['validity'] * 100 for r in res[m]])
             for m in ['Random', 'Sensible']}
        if d['Random'] and d['Sensible']:
            bp = ax.boxplot(
                [d['Random'], d['Sensible']],
                positions=[1, 2], widths=0.6, patch_artist=True,
                medianprops=dict(color='black', linewidth=1.5)
            )
            for patch, m in zip(bp['boxes'], ['Random', 'Sensible']):
                patch.set_facecolor(COLORS[m])
                patch.set_alpha(0.7)
            _, p = mannwhitneyu(d['Random'], d['Sensible'])
            ax.text(
                1.5, ax.get_ylim()[1] * 0.97,
                f'p={p:.4f} ({sig_stars(p)})',
                ha='center', fontsize=8, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3',
                          facecolor='lightyellow', alpha=0.8)
            )
        ax.set_xticks([1, 2])
        ax.set_xticklabels(['Random', 'Sensible'])
        ax.set_ylabel('Initial Validity Rate (%)')
        ax.set_title(pn)
        ax.set_ylim([0, 105])

    plt.tight_layout()
    plt.savefig('fig1_initial_quality.png')
    plt.close()
    print("\nSaved: fig1_initial_quality.png")

    # --- Fig 2: Convergence curves ---
    fig, axes = plt.subplots(n_probs, 2, figsize=(14, 4 * n_probs))
    fig.suptitle(
        'Convergence: Best MSE & Structural Diversity',
        fontsize=13, fontweight='bold', y=1.01
    )

    for row, (pn, res) in enumerate(all_results.items()):
        ng = len(res['Random'][0]['hist']['min'])
        x_ax = np.arange(ng)

        # Best MSE
        ax = axes[row, 0]
        for m in ['Random', 'Sensible']:
            c = np.array([r['hist']['min'] for r in res[m]], dtype=float)
            c = np.where(np.isnan(c) | (c > 1e8), np.nan, c)
            mn = np.nanmean(c, axis=0)
            sd = np.nanstd(c, axis=0)
            ax.plot(x_ax, mn, label=m, color=COLORS[m])
            ax.fill_between(x_ax, mn - sd, mn + sd,
                            color=COLORS[m], alpha=0.15)
        ax.set_xlabel('Generation')
        ax.set_ylabel('Best MSE (train)')
        ax.set_title(f'{pn}: Convergence')
        ax.legend(fontsize=8)

        # Structural diversity
        ax = axes[row, 1]
        for m in ['Random', 'Sensible']:
            c = np.array([r['hist']['struct_div'] for r in res[m]])
            mn = np.mean(c, axis=0)
            sd = np.std(c, axis=0)
            ax.plot(x_ax, mn, label=m, color=COLORS[m])
            ax.fill_between(x_ax, mn - sd, mn + sd,
                            color=COLORS[m], alpha=0.15)
        ax.set_xlabel('Generation')
        ax.set_ylabel('Structural Diversity')
        ax.set_title(f'{pn}: Diversity')
        ax.legend(fontsize=8)

    plt.tight_layout()
    plt.savefig('fig2_convergence.png')
    plt.close()
    print("Saved: fig2_convergence.png")

    # --- Fig 3: Final fitness boxplots ---
    fig, axes = plt.subplots(2, n_probs, figsize=(5 * n_probs, 10))
    fig.suptitle(
        f'Final Fitness Distribution ({N_RUNS} runs × {MAX_GENS} generations)',
        fontsize=13, fontweight='bold'
    )

    for col, (pn, res) in enumerate(all_results.items()):
        for row, (ylabel, key) in enumerate([
            ('Train MSE', lambda r: r['train_mse']),
            ('Test MSE', lambda r: r['test_mse']),
        ]):
            ax = axes[row, col]
            d = {m: safe_vals([key(r) for r in res[m]])
                 for m in ['Random', 'Sensible']}
            if d['Random'] and d['Sensible']:
                bp = ax.boxplot(
                    [d['Random'], d['Sensible']],
                    labels=['Random', 'Sensible'],
                    patch_artist=True,
                    medianprops=dict(color='black', linewidth=1.5)
                )
                for patch, m in zip(bp['boxes'], ['Random', 'Sensible']):
                    patch.set_facecolor(COLORS[m])
                    patch.set_alpha(0.7)
                _, p = mannwhitneyu(d['Random'], d['Sensible'])
                a_val = vda(d['Random'], d['Sensible'])
                ax.text(
                    1.5, ax.get_ylim()[1] * 0.97,
                    f'p={p:.4f} ({sig_stars(p)})\nA={a_val:.3f}',
                    ha='center', fontsize=8, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.3',
                              facecolor='lightyellow', alpha=0.8)
                )
            ax.set_ylabel(ylabel)
            if row == 0:
                ax.set_title(pn)

    plt.tight_layout()
    plt.savefig('fig3_boxplots.png')
    plt.close()
    print("Saved: fig3_boxplots.png")

    # --- Fig 4: Population structure over evolution ---
    fig, axes = plt.subplots(2, n_probs, figsize=(5 * n_probs, 7))
    fig.suptitle(
        'Population Structure Over Evolution',
        fontsize=13, fontweight='bold'
    )

    for col, (pn, res) in enumerate(all_results.items()):
        ng = len(res['Random'][0]['hist']['avg_depth'])
        x_ax = np.arange(ng)
        for row, (ylabel, key) in enumerate([
            ('Avg Tree Depth', 'avg_depth'),
            ('Avg Nodes', 'avg_nodes'),
        ]):
            ax = axes[row, col]
            for m in ['Random', 'Sensible']:
                c = np.array([r['hist'][key] for r in res[m]])
                mn = np.mean(c, axis=0)
                sd = np.std(c, axis=0)
                ax.plot(x_ax, mn, label=m, color=COLORS[m])
                ax.fill_between(x_ax, mn - sd, mn + sd,
                                color=COLORS[m], alpha=0.15)
            ax.set_xlabel('Generation')
            ax.set_ylabel(ylabel)
            ax.legend(fontsize=8)
            if row == 0:
                ax.set_title(pn)

    plt.tight_layout()
    plt.savefig('fig4_structure.png')
    plt.close()
    print("Saved: fig4_structure.png")

    # --- Fig 5: Invalid individuals over generations ---
    fig, axes = plt.subplots(1, n_probs, figsize=(5 * n_probs, 4))
    fig.suptitle(
        'Invalid Individuals Over Evolution',
        fontsize=13, fontweight='bold'
    )

    for col, (pn, res) in enumerate(all_results.items()):
        ax = axes[col] if n_probs > 1 else axes
        ng = len(res['Random'][0]['hist']['invalid'])
        x_ax = np.arange(ng)
        for m in ['Random', 'Sensible']:
            c = np.array(
                [r['hist']['invalid'] for r in res[m]], dtype=float
            )
            mn = np.mean(c, axis=0)
            sd = np.std(c, axis=0)
            ax.plot(x_ax, mn, label=m, color=COLORS[m])
            ax.fill_between(x_ax, mn - sd, mn + sd,
                            color=COLORS[m], alpha=0.15)
        ax.set_xlabel('Generation')
        ax.set_ylabel('# Invalid')
        ax.set_title(pn)
        ax.legend(fontsize=8)

    plt.tight_layout()
    plt.savefig('fig5_invalids.png')
    plt.close()
    print("Saved: fig5_invalids.png")

    # ==================================================================
    # 15. BEST SOLUTIONS & SAVE DATA
    # ==================================================================

    print("\n" + "=" * 70)
    print(" BEST SOLUTIONS FOUND")
    print("=" * 70)

    for pn, res in all_results.items():
        print(f"\n{'#' * 50}")
        print(f"# {pn}")
        print(f"{'#' * 50}")
        for m in ['Random', 'Sensible']:
            v = [r for r in res[m]
                 if r['train_mse'] is not None
                 and not np.isnan(r['train_mse'])]
            if v:
                b = min(v, key=lambda r: r['train_mse'])
                ph = (b['pheno'] or 'N/A')[:120]
                print(f"\n  {m}:")
                print(f"    Train MSE={b['train_mse']:.6f}  "
                      f"Test MSE={b['test_mse']:.6f}  "
                      f"Depth={b['depth']}  Nodes={b['nodes']}")
                print(f"    {ph}")

    # --- Save JSON ---
    def serialise(obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        if isinstance(obj, (np.floating, float)):
            return None if np.isnan(obj) else float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, dict):
            return {k: serialise(v) for k, v in obj.items()}
        if isinstance(obj, list):
            return [serialise(v) for v in obj]
        return obj

    with open('ge_results_raw.json', 'w') as f:
        json.dump(serialise(all_results), f)

    config = {
        'date': datetime.now().isoformat(),
        'source': 'GRAPE-faithful core with 4 benchmarks',
        'parameters': {
            'population_size': POP_SIZE,
            'max_generations': MAX_GENS,
            'p_crossover': P_CX,
            'p_mutation': P_MUT,
            'elite_size': ELITE,
            'tournament_size': TOURN,
            'max_tree_depth': MAX_TREE_DEPTH,
            'codon_size': CODON_SIZE,
            'codon_consumption': 'lazy',
            'fitness_metric': 'MSE',
            'random_init_genome_length': [MIN_INIT_GL, MAX_INIT_GL],
            'sensible_init_depth': [MIN_INIT_DEPTH, MAX_INIT_DEPTH],
            'sensible_method': 'Ramped Half-and-Half (RHH)',
        },
        'n_runs': N_RUNS,
        'seed': SEED,
        'total_seconds': total_t,
        'problems': list(PROBLEMS.keys()),
        'references': [
            'Ryan & Azad (2003) Sensible initialisation in GE, GECCO',
            'Nicolau (2017) Understanding GE: initialisation, GPEM 18(4)',
            'de Lima et al. (2022) GRAPE, Signals 3(3), 642-663',
            'Murphy et al. (2024) Structured GE Initialisation, GPEM 25(2)',
        ],
    }
    with open('experiment_config.json', 'w') as f:
        json.dump(config, f, indent=2)

    print(f"\n{'=' * 70}")
    print(f" ALL DONE — {total_t / 60:.1f} min total")
    print(f"{'=' * 70}")
    print(" Output files:")
    print("   fig1_initial_quality.png")
    print("   fig2_convergence.png")
    print("   fig3_boxplots.png")
    print("   fig4_structure.png")
    print("   fig5_invalids.png")
    print("   ge_results_raw.json")
    print("   experiment_config.json")
    print("\n EXPERIMENT COMPLETE!")


if __name__ == '__main__':
    main()

 GRAMMATICAL EVOLUTION: INITIALIZATION METHODS COMPARISON
 (GRAPE-faithful core · Sequential · 4 benchmarks)
Experiment date : 2026-02-11 20:25
GE library loaded
 EXPERIMENTAL PARAMETERS (GRAPE)
  Runs           : 30
  Generations    : 200
  Population     : 200
  Elite Size     : 0
  Tournament     : 7
  Crossover      : 0.8
  Mutation Rate  : 0.01
  Max Tree Depth : 35
  Codon Size     : 255
  Random genome  : [30, 50]
  Sensible depth : [3, 13]
  Mapper         : Lazy codon consumption
  Fitness        : MSE

Benchmark Problems:
  Keijzer-6: Harmonic series (1D)
  Nguyen-7: ln(x+1)+ln(x²+1) (1D)
  Pagie-1: 1/(1+x^-4)+1/(1+y^-4) (2D)
  Diabetes: Sklearn benchmark (10D)

 RUNNING EXPERIMENT (Sequential)
 4 Problems × 2 Methods × 30 Runs = 240 jobs

############################################################
# Keijzer-6: Harmonic series (1D)
############################################################
  Grammar: 2 NTs, 20 total productions
    <e>: 10 prods (recursive=8, non-recursive

KeyboardInterrupt: 